# Case study 2: spam email detection from ACIC (neural network)
This notebook includes experiments from Case Study 2 from the paper Multi-Source Causal Inference Using Control Variates. Specifically, this notebook contains experiments using the neural network model to estimate the ATE and odds ratios.

This experiment uses $n_2 = 10,000$ samples for the dataset without selection bias.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor

import data_sampler
import bootstrap

In [2]:
df_orig = pd.read_csv('spam_binMod11.csv', sep=",")

In [4]:
df_orig.describe()

,Y,A,V1,V2,V3,V4,V5,V6,V7,V8,...,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.311100,0.207700,0.313405,0.095635,0.110604,0.103395,0.08878,0.241847,0.058427,0.533474,...,0.142008,0.186700,1.656505,0.085842,0.802396,0.118049,0.100828,0.089238,0.532534,0.954270
std,0.462967,0.405681,0.694358,0.273618,0.387355,0.379405,0.26810,0.586613,0.197865,0.866654,...,0.455435,0.533876,1.759840,0.557759,1.181341,1.017302,0.332306,0.404872,1.633468,0.743763
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.464991
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.070000,...,0.000000,0.000000,1.310000,0.000000,0.230000,0.000000,0.000000,0.000000,0.000000,0.822859
75%,1.000000,0.000000,0.380000,0.000000,0.000000,0.000000,0.00000,0.170000,0.000000,0.770000,...,0.000000,0.000000,2.632500,0.000000,1.270000,0.000000,0.000000,0.000000,0.000000,1.312109
max,1.000000,1.000000,10.000000,5.880000,7.270000,11.110000,5.26000,11.110000,2.610000,9.670000,...,7.140000,9.090000,18.750000,18.180000,11.110000,17.100000,5.450000,12.500000,20.830000,7.005336


In [5]:
Y_COLUMN = 'Y'
Z_COLUMN = 'A'
X_COLUMNS = ['V%d' % i for i in range(1,23)]

# Load selection biased data

In [7]:
df_large = pd.read_csv('spam_binMod1_large1.csv', sep=",")

In [8]:
data_sampler_selection_bias = data_sampler.DataSampler(Z_COLUMN, X_COLUMNS, Y_COLUMN)
df_selection_bias = data_sampler_selection_bias.selection_bias_filter(df_large)
df_selection_bias = df_selection_bias.sample(3*len(df_orig), replace=False) 

In [9]:
df_selection_bias.describe()

,Y,A,V1,V2,V3,V4,V5,V6,V7,V8,...,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.801167,0.183800,0.180599,0.052648,0.036636,0.054533,0.048682,0.193934,0.030047,0.598244,...,0.080952,0.113648,1.190243,0.029462,0.452818,0.044702,0.032832,0.031519,1.318157,0.742718
std,0.399129,0.387327,0.506530,0.196100,0.222939,0.264764,0.198154,0.599005,0.147875,1.000107,...,0.298130,0.408723,1.616149,0.270355,0.906951,0.600110,0.192489,0.242769,2.462085,0.564568
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.390690
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.685367
75%,1.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.860000,...,0.000000,0.000000,1.880000,0.000000,0.630000,0.000000,0.000000,0.000000,1.820000,1.002101
max,1.000000,1.000000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,2.610000,9.670000,...,5.120000,9.090000,18.750000,18.180000,11.110000,17.100000,4.760000,12.500000,20.830000,6.929027


# Varying coefficient model with neural network function approximation

In this section, we assume the data is generated from 

$$P(Y=1 | Z = z, X = x) = \frac{e^{\beta_0^x + \beta_1^x z}}{ 1 + e^{\beta_0^x + \beta_1^xz}}$$

where $\beta_0^x, \beta_1^x$ are functions of $x$:
$$\beta_0^x = f_0(x; \theta_0), \quad \beta_1^x = f_1(x; \theta_1)$$

If we were to assume $\beta_0^x, \beta_1^x$ were linear in $x$, this would be equivalent to the above logistic regression model with explicit interaction terms.

To model more complicated interactions, we allow $f_0(x; \theta_0), f_1(x; \theta_1)$ to be nonlinear in $x$, and set these to be generated from a two layer neural network.

## Choose hyperparameters using 5-fold cross validation

In [10]:
data_sampler_nn = data_sampler.DataSamplerNN(Z_COLUMN, X_COLUMNS, Y_COLUMN)

In [ ]:
# Optimize for best hyperparams using 5-fold cross validation. 
num_hidden_layers_list = [4, 8]
hidden_dim_list = [4, 8, 16, 32]

kf = KFold(n_splits = 5, shuffle = True, random_state = 0)
    
best_accuracy = 0.0
best_hidden_dim = None
best_num_hidden_layers = None
for num_hidden_layers in num_hidden_layers_list:
    for hidden_dim in hidden_dim_list:
        print("Optimizing with num_hidden_layers=%d, hidden_dim=%d" % (num_hidden_layers, hidden_dim))
        CV_accuracies = []
        for train_index, test_index in kf.split(df_orig):
            df_train = df_orig.iloc[train_index]
            df_test =  df_orig.iloc[test_index]
            _ = data_sampler_nn.fit_outcome(df_train, 
                                            num_hidden_layers=num_hidden_layers, 
                                            hidden_dim=hidden_dim, 
                                            batch_size=10000, 
                                            epochs=1000, 
                                            step_size=0.001, 
                                            verbose=0, 
                                            print_metrics=False)
            accuracy = data_sampler_nn.print_metrics(df_test)
            CV_accuracies.append(accuracy)
        avg_CV_accuracy = np.mean(CV_accuracies)
        print("average CV accuracy:", avg_CV_accuracy)
        if avg_CV_accuracy > best_accuracy: 
            best_num_hidden_layers = num_hidden_layers
            best_hidden_dim = hidden_dim
            best_accuracy = avg_CV_accuracy

In [ ]:
print("best cross validation accuracy:", best_accuracy)
print("best hidden_dim:", best_hidden_dim)
print("best num_hidden_layers:", best_num_hidden_layers)

# Compute ATE estimates with and without control variate

In [86]:
def ATE_estimator_fn_interaction(df_input):
    data_sampler_nn = data_sampler.DataSamplerNN(Z_COLUMN, X_COLUMNS, Y_COLUMN)
    data_sampler_nn.fit_outcome(df_input, 
                                num_hidden_layers=8, 
                                hidden_dim=4, 
                                batch_size=10000, 
                                epochs=1000, 
                                step_size=0.001, 
                                verbose=0, 
                                print_metrics=False)
    return data_sampler_nn.get_ATE_estimate(df_input)

def CV_estimator_fn_interaction(df_input_obs, df_input_bias):
    data_sampler_nn = data_sampler.DataSamplerNN(Z_COLUMN, X_COLUMNS, Y_COLUMN)
    OR_xs = df_input_obs[X_COLUMNS] # Average over all xs in the observational dataset.
    # Estimate OR from observational dataset
    data_sampler_nn.fit_outcome(df_input_obs, 
                                num_hidden_layers=8, 
                                hidden_dim=4, 
                                batch_size=10000, 
                                epochs=1000, 
                                step_size=0.001, 
                                verbose=0, 
                                print_metrics=False)
    OR_obs = np.mean(data_sampler_nn.get_conditional_OR_estimates(OR_xs))
    # Estimate OR from selection bias dataset
    data_sampler_nn.fit_outcome(df_input_bias, 
                                num_hidden_layers=8, 
                                hidden_dim=4, 
                                batch_size=10000, 
                                epochs=1000, 
                                step_size=0.001, 
                                verbose=0, 
                                print_metrics=False)
    OR_bias = np.mean(data_sampler_nn.get_conditional_OR_estimates(OR_xs))
    return OR_obs - OR_bias

CV_samples, ATE_hat_samples, _ = bootstrap.run_bootstrap_df(df_obs=df_orig, 
              df_bias=df_selection_bias, 
              n_replicates=300, 
              ATE_estimator_fn=ATE_estimator_fn_interaction,
              CV_estimator_fn=CV_estimator_fn_interaction,
             )

In [89]:
sample_cov = np.cov(np.array([ATE_hat_samples, CV_samples]), ddof=1)

# Get optimal control variates coefficient
cov_ATE_CV = sample_cov[0][1]
var_CV = sample_cov[1][1]
optimal_CV_coeff = cov_ATE_CV / var_CV
print("optimal CV coefficient:", optimal_CV_coeff)

optimal CV coefficient: -0.0006293352942657389


In [90]:
# Get variance/bias of ATE estimators with and without CV.
CV_samples, ATE_hat_samples, ATE_hat_CV_samples = bootstrap.run_bootstrap_df(
    df_obs=df_orig, 
    df_bias=df_selection_bias, 
    n_replicates=300, # Try increasing this
    ATE_estimator_fn=ATE_estimator_fn_interaction,
    CV_estimator_fn=CV_estimator_fn_interaction,
    optimal_CV_coeff=optimal_CV_coeff)

In [91]:
ATE_true = 0.106286795757474

ATE_var = np.var(np.array(ATE_hat_samples), ddof=1)
print(">>> Variance of ATE estimator:", ATE_var)

ATE_bias = np.mean(np.array(ATE_hat_samples)) - ATE_true
print(">>> Bias of ATE estimator:", ATE_bias)

ATE_CV_var = np.var(np.array(ATE_hat_CV_samples), ddof=1)
print(">>> Variance of ATE estimator with CV:", ATE_CV_var)

ATE_CV_bias = np.mean(np.array(ATE_hat_CV_samples)) - ATE_true
print(">>> Bias of ATE estimator with CV:", ATE_CV_bias)

>>> Variance of ATE estimator: 0.00047615127
>>> Bias of ATE estimator: 0.00034268764430135656
>>> Variance of ATE estimator with CV: 0.0004478699230335257
>>> Bias of ATE estimator with CV: 0.0013608659340094975
